# Day 23: Safe Cracking

## Part One

This is one of the top floors of the nicest tower in EBHQ. The Easter Bunny's private office is here, complete with a safe hidden behind a painting, and who wouldn't hide a star in a safe behind a painting?

The safe has a digital screen and keypad for code entry. A sticky note attached to the safe has a password hint on it: "eggs". The painting is of a large rabbit coloring some eggs. You see `7`.

When you go to type the code, though, nothing appears on the display; instead, the keypad comes apart in your hands, apparently having been smashed. Behind it is some kind of socket - one that matches a connector in your prototype computer! You pull apart the smashed keypad and extract the logic circuit, plug it into your computer, and plug your computer into the safe.

Now, you just need to figure out what output the keypad would have sent to the safe. You extract the assembunny code from the logic chip (your puzzle input).
The code looks like it uses almost the same architecture and instruction set that the monorail computer used! You should be able to use the same assembunny interpreter for this as you did there, but with one new instruction:

`tgl x` toggles the instruction `x` away (pointing at instructions like `jnz` does: positive means forward; negative means backward):

* For one-argument instructions, `inc` becomes `dec`, and all other one-argument instructions become `inc`.

* For two-argument instructions, `jnz` becomes `cpy`, and all other two-instructions become `jnz`.

* The arguments of a toggled instruction are not affected.

* If an attempt is made to toggle an instruction outside the program, nothing happens.

* If toggling produces an invalid instruction (like `cpy 1 2`) and an attempt is later made to execute that instruction, skip it instead.

* If `tgl` toggles itself (for example, if `a` is `0`, `tgl a` would target itself and become `inc a`), the resulting instruction is not executed until the next time it is reached.

For example, given this program:

```
cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
```

* `cpy 2 a` initializes register `a` to `2`.

* The first `tgl a` toggles an instruction `a` (`2`) away from it, which changes the third `tgl a` into `inc a`.

* The second `tgl a` also modifies an instruction `2` away from it, which changes the `cpy 1` a into `jnz 1 a`.

* The fourth line, which is now `inc a`, increments `a` to `3`.

* Finally, the fifth line, which is now `jnz 1 a`, jumps `a` (`3`) instructions ahead, skipping the `dec a` instructions.

In this example, the final value in register `a` is `3`.

The rest of the electronics seem to place the keypad entry (the number of eggs, `7`) in register `a`, run the code, and then send the value left in register `a` to the safe.

What value should be sent to the safe?

---

In [6]:
test = """cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a"""


# Initialise
inputs = [i[:-1].split(' ') for i in open('Day23.in').readlines()]
# inputs = [i.split(' ') for i in test.split('\n')]

register = {
    char: 0 for char in ('a', 'b', 'c', 'd')
}

register['a'] = 7

In [7]:
# Tweak the code to include tgl
def assembunny(reg, cmds, cursor=0):
    """
    Takes a register and commands then parses and runs the commands. It assumes cursor 
    starts at 0 but may be set as a option.
    
    :returns: None
    """
    # This function will parse a command (from cmds) at the given cursor position
    def parse(cursor):
        cmd, x, y, *_ = cmds[cursor] + [None]
        if cmd == 'cpy':
            if x in reg:
                reg[y] = reg[x]
            else:
                reg[y] = int(x)
        elif cmd == 'jnz':
            # Put -1 in here because we'll increment the cursor at the end
            y = reg[y] if y in reg else int(y)
            if delta:= optimise_jnz(reg, cmds[cursor + y:cursor], x):
                for char in reg:
                    reg[char] += delta[char]
            elif x in reg:
                cursor += y - 1 if reg[x] else 0
            else:
                cursor += y - 1 if int(x) else 0
        elif cmd == 'tgl':
            x = reg[x] if x in reg else int(x)
            tgl, a, b, *_ = cmds[cursor + x] + [None]
            if b is None:
                cmds[cursor + x] = [('dec' if tgl == 'inc' else 'inc'), a]
            else:
                cmds[cursor + x] = [('cpy' if tgl == 'jnz' else 'jnz'), a, b]
        else:
            reg[x] += 1 if cmd == 'inc' else -1
        return cursor + 1
    
    cursors = []
    high_cursor = 0
    
    # Run cmds until cursor exceeds len(cmds)
    while cursor < len(cmds):
#         cursors.append(f"{cursor}: {str(cmds[cursor]).ljust(18)} {reg}")
#         clear_output(wait=True)
#         print(f'highest cursor value: {high_cursor}\n' + '\n'.join(cursors))
#         time.sleep(0.1)
        try:
            cursor = parse(cursor)
        except:
#             print("WOOPS")
            cursor += 1
        
        if not reg['a'] % 10_000:
            clear_output(wait=True)
            print(reg)
#         if len(cursors) > 10:
#             cursors = cursors[1:]
#         if cursor > high_cursor:
#             high_cursor = cursor

def optimise_jnz(reg, cmds, jnz_char):
    """
    Given repeating arguments prior to a jnz optimize into one step if possible, else return None
    
    If possible, return delta as {a: delta_a, ..., d: delta_d}
    """
    if jnz_char not in ('a', 'b', 'c', 'd'):
        return None
    
    delta = {
        char: 0 for char in ('a', 'b', 'c', 'd')
    }
    
    if not [cmd for cmd in cmds if cmd[0] in ('tgl', 'jnz', 'cpy')]:
        for cmd in cmds:
            if cmd[0] == 'inc':
                delta[cmd[1]] += 1
            else:
                delta[cmd[1]] -= 1
    
        jnz_val = delta[jnz_char]
        
        return {
            char: int(delta[char]*reg[jnz_char]/(-jnz_val)) for char in delta
        }
        

# Run inputs through assumbunny
assembunny(register, inputs)

# Solution
print(f"The value {register['a']:,} is left in register a.")

{'a': 0, 'b': 2, 'c': 2, 'd': 2520}
The value 11,340 is left in register a.


In [140]:
optimise_jnz(register, inputs[5:7], 'c')

{'a': 35, 'b': 0, 'c': -35, 'd': 0}

In [136]:
inputs[5:7]

[['inc', 'a'], ['dec', 'c']]

In [4]:
from IPython.display import clear_output, display
import time

In [8]:
# Initialise
inputs = [i[:-1].split(' ') for i in open('Day23.in').readlines()]
# inputs = [i.split(' ') for i in test.split('\n')]

register = {
    char: 0 for char in ('a', 'b', 'c', 'd')
}

register['a'] = 12

# Run inputs through assumbunny
assembunny(register, inputs)

# Solution
print(f"The value {register['a']:,} is left in register a.")

{'a': 479000000, 'b': 2, 'c': 2, 'd': 800}
The value 479,007,900 is left in register a.
